In [127]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tensorflow as tf

from skimage import color, exposure, filters, io, morphology, util
from math import sqrt
from numpy import loadtxt
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense

In [90]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

### A. _Fully Connected Neural Networks_
1. Lakukan klasifikasi untuk dataset cifar menggunakan **_Fully Connected Neural Networks_** dengan arsitektur dasar sebagai berikut:
    - _Fully Connected Layer_ ($1000$ _hidden node_).
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Fully Connected Layer_ ($1000$ _hidden node_)
    - _Activation Layer_ ($\text{ReLU function}$).
    - _Output Layer_ (10 kelas).
    - _Activation Layer_ ($\text{Softmax function}$).
    - _Classification Result_.

Fitur yang digunakan bebas (_handcrafted_/_non-convolution_). Hitung metrik ($\text{accuracy}$, $\text{sensitivity}$, $\text{specificity}$, dan $\text{f1-score}$) untuk data train dan data test.

In [136]:
## Fungsi Bantuan
def f1_m(y_true, y_pred): # F1 Score
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

def get_best_class(x):
  return K.argmax(x)

In [137]:
## Melakukan inisialisasi node per layer
nodes_per_layer = [1000, 1000, 10]
activation_function_per_layer = [tf.nn.relu, tf.nn.relu, tf.nn.softmax]

## Inisialisasi Layer Model Parameter
layers = []
layers.append(tf.keras.layers.Flatten())

## Menyisipkan parameter node dan fungsi aktivasinya
for i in range(len(nodes_per_layer)):
  node = nodes_per_layer[i]
  activation = activation_function_per_layer[i]
  layers.append(keras.layers.Dense(node, activation=activation))

## Melakukan klasifikasi
layers.append(keras.layers.Dense(1, activation=get_best_class))

## Mendefinisikan model
model = Sequential(layers)

## Membangun model
model.build(input_shape=x_train.shape)
model.output_shape

(50000,)

In [138]:
## Melakukan kompilasi ke dalam model
model.compile(
    optimizer=tf.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=[
             tf.keras.metrics.Accuracy(name='accuracy'),
             tf.keras.metrics.SensitivityAtSpecificity(0.5),
             tf.keras.metrics.SpecificityAtSensitivity(0.5)
    ])

In [139]:
## Melakukan fitting
beginning_hist = model.fit(x_train, y_train, epochs=20, batch_size=2000, validation_data=(x_test,y_test), use_multiprocessing=True)

Epoch 1/20


TypeError: ignored